In [1]:
import torch
import torch.nn as nn

X1 = torch.randint(0, 10, (1, 10, 1)).float()
X2 = torch.randint(0, 10, (1, 7, 1)).float()

rnn = nn.RNN(input_size=1, hidden_size=1, num_layers=1, batch_first=True)
output, hn = rnn(X1)
print(output.shape, hn.shape)
(
    output[:, -1, :],
    hn[:, -1, :],
)  ## Es decir, el último hidden state es igual a la última salida

torch.Size([1, 10, 1]) torch.Size([1, 1, 1])


(tensor([[0.9993]], grad_fn=<SelectBackward0>),
 tensor([[0.9993]], grad_fn=<SelectBackward0>))

In [2]:
torch.manual_seed(42)
x1 = torch.randint(0, 10, (1, 3, 1)).float()
x2 = torch.randint(0, 10, (1, 2, 1)).float()
x1, x2

(tensor([[[2.],
          [7.],
          [6.]]]),
 tensor([[[4.],
          [6.]]]))

In [3]:
rnn = nn.RNN(input_size=1, hidden_size=2, num_layers=1, batch_first=True)


def forward_pass_rnn(x, hidden_size=2):
    N, L, D = x.shape
    h = [torch.zeros(1, hidden_size)]
    for seq_id in range(L):
        h.append(
            torch.tanh(
                x[0, seq_id, :] @ rnn.weight_ih_l0.T
                + rnn.bias_ih_l0
                + h[-1] @ rnn.weight_hh_l0.T
                + rnn.bias_hh_l0
            ).detach()
        )
    return h[1:]


forward_pass_rnn(x1)

[tensor([[0.4727, 0.1731]]),
 tensor([[ 0.9106, -0.9335]]),
 tensor([[ 0.7067, -0.9780]])]

In [13]:
forward_pass_rnn(x2)

[tensor([[ 0.6634, -0.4728]]), tensor([[ 0.7876, -0.9505]])]

In [12]:
torch.manual_seed(42)


class POSTaggingRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True,
        )
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        rnn_out, hn = self.rnn(x)
        print("Tamaño del Output de RNN: ", hn.shape)
        logits = self.fc(rnn_out)
        return logits


model = POSTaggingRNN(input_size=1, hidden_size=4, output_size=3)
output = model(x2)
print("Shape del Output Final: ", output.shape)
output

Tamaño del Output de RNN:  torch.Size([1, 1, 4])
Shape del Output Final:  torch.Size([1, 2, 3])


tensor([[[-0.9738,  0.6183,  0.4344],
         [-1.0537,  0.7482,  0.4776]]], grad_fn=<ViewBackward0>)

In [14]:
torch.manual_seed(42)


class SentimentAnalysisRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True,
        )
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        rnn_out, hn = self.rnn(x)
        print("Tamaño del Hidden State del RNN: ", hn.shape)
        logits = self.fc(hn)
        return logits


model = SentimentAnalysisRNN(input_size=1, hidden_size=4, output_size=3)
output = torch.softmax(model(x2), dim=-1)
print("Shape del Output Final: ", output.shape)
output

Tamaño del Hidden State del RNN:  torch.Size([1, 1, 4])
Shape del Output Final:  torch.Size([1, 1, 3])


tensor([[[0.0856, 0.5187, 0.3957]]], grad_fn=<SoftmaxBackward0>)